In [1]:
from pathlib import Path
from uuid import uuid4

import yaml
import numpy as np
from alr_sim.core import Scene

from alr_sim_tools.ycb_utils import YCBLoader
from alr_sim_tools.scene_utils import record_camera_data, reset_scene
from alr_sim_tools.visualization_utils import plot_pointcloud

pybullet build time: May  2 2023 05:55:04


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
FACTORY_STRING = "mj_beta"
OBJECT_POS = (0.5, 0.0, 0.2)
CAM_POS = (0.5, 0.0, 1)
CAM_QUAT = [0.7071067811865476, 0.0, 0.0, -0.7071067811865475]
CAM_HEIGHT = 480
CAM_WIDTH = 640
INITIAL_ROBOT_POS = (0.0, 0.5, 0.2)
INITIAL_ROBOT_QUAT = (0, 1, 0, 0)
RENDER_MODE = Scene.RenderMode.HUMAN    

YCB_FOLDER = Path.home() / "Documents" / "SF-ObjectDataset" / "YCB"
YCB_OBJECT_ID = "011_banana"

DATATYPE = "pointcloud_segmented_points"
DATA_TARGET_FOLDER = Path.cwd().parent / "data" / DATATYPE / YCB_OBJECT_ID

In [3]:
ycb_loader = YCBLoader(ycb_base_folder=YCB_FOLDER, factory_string="mj_beta")

sim_obj = ycb_loader.get_ycb_object(
    pos=OBJECT_POS, quat=(0, 1, 0, 0), obj_id=YCB_OBJECT_ID, name=YCB_OBJECT_ID, grounded=True
)

In [4]:
camera_data, scene, agent = record_camera_data(
    factory_string=FACTORY_STRING,
    cam_pos=CAM_POS,
    cam_quat=CAM_QUAT,
    cam_height=CAM_HEIGHT,
    cam_width=CAM_WIDTH,
    robot_pos=INITIAL_ROBOT_POS,
    robot_quat=INITIAL_ROBOT_QUAT,
    object_list=[sim_obj],
    target_obj_name=YCB_OBJECT_ID,
    render_mode=RENDER_MODE,
    wait_time=1,
)

reset_scene(FACTORY_STRING, scene, agent)

In [5]:
# plot_pointcloud(camera_data.pointcloud_points, camera_data.pointcloud_colors, "open3d")

metadata = {
    "factory_string": FACTORY_STRING,
    "cam_pos": CAM_POS,
    "cam_quat": CAM_QUAT,
    "cam_height": CAM_HEIGHT,
    "cam_width": CAM_WIDTH,
    "object_pos": OBJECT_POS,
    "object_quat": (0, 1, 0, 0),
    "object_id": YCB_OBJECT_ID,
}
file_id = uuid4()

if not DATA_TARGET_FOLDER.exists():
    DATA_TARGET_FOLDER.mkdir(parents=True)

data = camera_data.__getattribute__(DATATYPE)

np.save(DATA_TARGET_FOLDER / f"{file_id}.npy", data)
with open(DATA_TARGET_FOLDER / f"{file_id}.yaml", "w") as f:
    yaml.dump(metadata, f)